### Load tensorflow

In [0]:
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
ls -l

total 8
drwx------ 3 root root 4096 Mar 17 08:52 gdrive/
drwxr-xr-x 1 root root 4096 Mar  8 17:26 sample_data/


### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/prices.csv')

### Check all columns in the dataset

In [6]:
data.dtypes

date       object
symbol     object
open      float64
close     float64
low       float64
high      float64
volume    float64
dtype: object

In [7]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Drop columns `date` and  `symbol`

In [0]:
data_org = data

In [0]:
data.drop(['date','symbol'],axis = 1,inplace=True)

In [10]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [11]:
data.shape

(851264, 5)

### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data_1000 = data.iloc[0:1000,]

In [13]:
data_1000.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [14]:
data_1000.shape

(1000, 5)

### Divide the data into train and test sets

In [0]:
X = data_1000[['open','low','high','volume']]

In [41]:
X.head()

,open,low,high,volume
0,123.430000,122.309998,126.250000,2163600.0
1,125.239998,119.940002,125.540001,2386400.0
2,116.379997,114.930000,119.739998,2489500.0
3,115.480003,113.500000,117.440002,2006300.0
4,117.010002,114.089996,117.330002,1408600.0


In [0]:
Y = data_1000['close']

In [42]:
Y.head()

0    125.839996
1    119.980003
2    114.949997
3    116.620003
4    114.970001
Name: close, dtype: float64

### Split Data

In [0]:
from sklearn.model_selection import train_test_split

### Split the Dataset which has been reduced by PCA.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
x = tf.placeholder(tf.float32, shape=(None, 4))
x_nn = tf.nn.l2_normalize(x)
y = tf.placeholder(tf.float32, shape=(None))

2.Define Weights and Bias

In [20]:
#We are initializing weights and Bias with Zero
w = tf.Variable(tf.zeros(shape=[4,1]))
b = tf.Variable(tf.zeros(shape=[1]))

Instructions for updating:
Colocations handled automatically by placer.


3.Prediction

In [0]:
y_ = tf.add(tf.matmul(x_nn, w), b)

4.Loss (Cost) Function [Mean square error]

In [0]:
mseavg = tf.reduce_mean(tf.square(y - y_))

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [23]:
optimiser = tf.train.GradientDescentOptimizer(0.01).minimize(mseavg)

Instructions for updating:
Use tf.cast instead.


## Execute the Graph for 100 epochs and observe the loss

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
epochs = 100

In [25]:
for epoch in range(epochs):
  opt,train_loss = sess.run([optimiser,mseavg],feed_dict={x:X_train,y:y_train})
  # print(epoch)
  if epoch / 10 == 0:
    print('Current Loss on iteration :',train_loss)

('Current Loss on iteration :', 8370.582)
('Current Loss on iteration :', 8184.713)
('Current Loss on iteration :', 8006.204)
('Current Loss on iteration :', 7834.7666)
('Current Loss on iteration :', 7670.118)
('Current Loss on iteration :', 7511.9907)
('Current Loss on iteration :', 7360.1265)
('Current Loss on iteration :', 7214.2764)
('Current Loss on iteration :', 7074.2036)
('Current Loss on iteration :', 6939.678)


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [26]:
sess.run(w)

array([[8.7868657e-06],
       [8.6985820e-06],
       [8.8714669e-06],
       [7.1968395e-01]], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [27]:
print(train_loss)

3763.4832


In [28]:
for epoch in range(epochs):
  opt,test_loss = sess.run([optimiser,mseavg],feed_dict={x:X_test,y:y_test})
  # print(epoch)
  if epoch / 10 == 0:
    print('Current Loss on iteration :',train_loss)

('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)
('Current Loss on iteration :', 3763.4832)


In [29]:
diff = train_loss-test_loss
print(diff)

1025.8232


In [30]:
sess.close

<bound method Session.close of <tensorflow.python.client.session.Session object at 0x7f53732a6bd0>>

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))


#Add Dense Layer which provides 1 Outputs 
model.add(tf.keras.layers.Dense(1))


### Execute the model

In [47]:
#Comile the model
model.compile(optimizer='sgd', loss='mse')

Instructions for updating:
Use tf.cast instead.


### Classification using Keras 

In [48]:
model.fit(X, Y, epochs=100)

Epoch 1/100
1000/1000 [==============================] - 0s 303us/sample - loss: nan
Epoch 2/100
1000/1000 [==============================] - 0s 100us/sample - loss: nan
Epoch 3/100
1000/1000 [==============================] - 0s 106us/sample - loss: nan
Epoch 4/100
1000/1000 [==============================] - 0s 112us/sample - loss: nan
Epoch 5/100
1000/1000 [==============================] - 0s 109us/sample - loss: nan
Epoch 6/100
1000/1000 [==============================] - 0s 108us/sample - loss: nan
Epoch 7/100
1000/1000 [==============================] - 0s 109us/sample - loss: nan
Epoch 8/100
1000/1000 [==============================] - 0s 101us/sample - loss: nan
Epoch 9/100
1000/1000 [==============================] - 0s 103us/sample - loss: nan
Epoch 10/100
1000/1000 [==============================] - 0s 104us/sample - loss: nan
Epoch 11/100
1000/1000 [==============================] - 0s 106us/sample - loss: nan
Epoch 12/100
1000/1000 [==============================] - 0s 10

### Load the given Iris data using pandas (Iris.csv)

In [0]:
irisdata = pd.read_csv('/content/gdrive/My Drive/Iris-1.csv')

### Splitting the data into feature set and target set

In [111]:
irisdata.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
X = irisdata[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
Y = irisdata['Species']

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
Y = pd.get_dummies(Y)

In [70]:
Y.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split

### Split the Dataset which has been reduced by PCA.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

In [0]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10,input_shape=(4,)))

In [0]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8))

In [0]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

### Fitting the model and predicting 

In [95]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=30,
          batch_size=20)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 0s 393us/sample - loss: 0.2588 - acc: 0.8667 - val_loss: 0.6417 - val_acc: 0.7111
Epoch 2/30
105/105 [==============================] - 0s 388us/sample - loss: 0.1542 - acc: 0.9524 - val_loss: 0.5681 - val_acc: 0.7111
Epoch 3/30
105/105 [==============================] - 0s 388us/sample - loss: 0.2033 - acc: 0.9333 - val_loss: 0.6018 - val_acc: 0.7111
Epoch 4/30
105/105 [==============================] - 0s 361us/sample - loss: 0.1416 - acc: 0.9429 - val_loss: 0.6276 - val_acc: 0.7111
Epoch 5/30
105/105 [==============================] - 0s 365us/sample - loss: 0.1693 - acc: 0.9429 - val_loss: 0.5288 - val_acc: 0.7556
Epoch 6/30
105/105 [==============================] - 0s 377us/sample - loss: 0.1870 - acc: 0.9429 - val_loss: 0.4828 - val_acc: 0.8000
Epoch 7/30
105/105 [==============================] - 0s 401us/sample - loss: 0.2559 - acc: 0.8952 - val_loss: 0.3887 - val_acc: 0.8000
Epo

### Report Accuracy of the predicted values

In [0]:
prediction = model.predict(X_test)

In [0]:
import numpy as np

y_predict = np.argmax(prediction,axis=1)

In [0]:
y_actual = np.argmax(y_test.values,axis=1)

In [120]:
y_actual

array([2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 2,
       1])

In [121]:
y_predict

array([2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 1, 2,
       1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 2,
       1])

In [124]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(y_actual, y_predict) 
print 'Confusion Matrix :'
print(results) 

 Confusion Matrix :
[[12  0  0]
 [ 0 13  3]
 [ 0  1 16]]
